# Create chips for the majority of GB

This notebook samples from the majority of GB and dumps them to disk as TIF files to be loaded by TensorFlow.

In [34]:
import glob
import os
import shutil
import warnings

from itertools import product
from pathlib import Path

import contextily
import dask
import dask_geopandas
import dask.bag
import dask.dataframe
import geopandas
import numpy
import pandas
import pygeos
import pyogrio
import xarray, rioxarray

from dask_geopandas.hilbert_distance import _hilbert_distance
from dask.distributed import Client, LocalCluster
from shapely.geometry import box
from tqdm.auto import tqdm

import tools

In [2]:
client = Client(
    LocalCluster(n_workers=16, threads_per_worker=1)
)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 16,Total memory: 125.54 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37099,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 125.54 GiB
Comm: tcp://172.17.0.3:43223,Total threads: 1
Dashboard: http://172.17.0.3:43087/status,Memory: 7.85 GiB
Nanny: tcp://127.0.0.1:36951,


## Specs

In [3]:
specs = {
    'chip_size': 32,
    'bands': [1, 2, 3], #RGB
    'mosaic_p': (
        '/home/jovyan/work/urbangrammar_samba/'
        'ghs_composite_s2/GHS-composite-S2.vrt'
    ),
    'spsig_p': (
        '/home/jovyan/work/urbangrammar_samba/spatial_signatures/'
        'signatures/'
        'signatures_combined_levels_simplified.gpkg'
    ),
    'points_temp': '/home/jovyan/work/chips_gb/temp/points/',
    'folder': (
        '/home/jovyan/work/chips_gb/32/'
    ),
}

## Load signatures and Sentinel2 mosaic

In [ ]:
%%time
spsig = pyogrio.read_dataframe(specs['spsig_p'])

In [ ]:
mosaic = rioxarray.open_rasterio(
    specs['mosaic_p'], chunks={'x': 1024, 'y': 1024}
)

Get range of coordinates from the mosaic

In [ ]:
start_x = float(mosaic["x"].min())
start_y = float(mosaic["y"].min())
end_x = float(mosaic["x"].max())
end_y = float(mosaic["y"].max())

Create coordinates of individual points at set chip size. We are using only the southern half of the extent that covers the country up to Perth.

In [ ]:
x_coords = numpy.arange(start_x, end_x, specs["chip_size"])
y_coords = numpy.arange(start_y, end_y, specs["chip_size"])

In [ ]:
x_bag = dask.bag.from_sequence(x_coords)
y_bag = dask.bag.from_sequence(y_coords[:y_coords.shape[0]//2])

In [ ]:
product_bag = x_bag.product(y_bag)

In [ ]:
ddf = product_bag.to_dataframe(meta={"x": float, "y":float})

In [ ]:
%%time
ddf.to_parquet("/home/jovyan/work/chips_gb/temp/coords/")

Create points from coordinates

In [ ]:
ddf = dask.dataframe.read_parquet("/home/jovyan/work/chips_gb/temp/coords/")

In [ ]:
ddf["geometry"] = dask_geopandas.points_from_xy(ddf, "x", "y", crs=27700)

In [ ]:
gddf = dask_geopandas.from_dask_dataframe(ddf).set_crs(27700)

In [ ]:
gddf.to_parquet(specs["points_temp"], overwrite=True)

Filter points that fall on land.

In [ ]:
signatures = spsig[['signature_type', 'geometry']].set_index(_hilbert_distance(spsig, spsig.total_bounds, p=10))

In [ ]:
signatures = dask_geopandas.from_geopandas(signatures.sort_index(), npartitions=1000)
signatures.calculate_spatial_partitions()

In [ ]:
points = dask_geopandas.read_parquet(specs["points_temp"])

In [ ]:
points_within = dask_geopandas.sjoin(points, signatures, how="inner", op='within')

In [ ]:
%%time
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
points_within.repartition(npartitions=1000).to_parquet("/home/jovyan/work/chips_gb/temp/points_within/")

In [ ]:
client.restart()

Create chip bounds polygons.

In [ ]:
points_within = dask.dataframe.read_parquet("/home/jovyan/work/chips_gb/temp/points_within/", columns=["geometry"])

In [ ]:
points_within["geometry"] = points_within["geometry"].map_partitions(geopandas.GeoSeries.from_wkb, meta=geopandas.GeoSeries())

In [ ]:
points_within = dask_geopandas.from_dask_dataframe(points_within)

In [ ]:
polygons = points_within.buffer(specs['chip_size'] / 2, cap_style=3)

In [ ]:
points_within["geometry"] = polygons

In [ ]:
points_within = points_within.repartition(partition_size="5MB")

In [ ]:
points_within.to_parquet("/home/jovyan/work/chips_gb/temp/polygons/", overwrite=True)

Dump signatures to sorted partitioned parquet.

In [ ]:
signatures = spsig[['signature_type', 'geometry']].set_index(_hilbert_distance(spsig, spsig.total_bounds, p=10))

In [ ]:
signatures = dask_geopandas.from_geopandas(signatures.sort_index(), npartitions=1000)
signatures.to_parquet("/home/jovyan/work/chips_gb/temp/signatures")

In [ ]:
client.restart()

Filter polygons fully wihtin a single signature type.

In [ ]:
polygons = dask_geopandas.read_parquet("/home/jovyan/work/chips_gb/temp/polygons/").set_crs(27700)
signatures = dask_geopandas.read_parquet("/home/jovyan/work/chips_gb/temp/signatures/")

In [ ]:
polygons_within = dask_geopandas.sjoin(polygons, spsig[['signature_type', 'geometry']], op="within")

In [ ]:
polygons_within.to_parquet("/home/jovyan/work/chips_gb/temp/chip_bounds/")

Get sample of max 50k chips per type.

In [ ]:
chip_bds = dask_geopandas.read_parquet("/home/jovyan/work/chips_gb/temp/chip_bounds/", columns=["geometry", "signature_type"])

In [7]:
types = ['0_0', '1_0', '3_0', '4_0', '5_0', '6_0', '7_0', '8_0', '2_0', '2_1', '2_2', '9_0', '9_1', '9_2', '9_4', '9_5']

In [ ]:
for t in types:
    df = chip_bds[chip_bds["signature_type"] == t].compute()
    n = 50_000 if len(df) > 50_000 else len(df)
    df.sample(n=n, random_state=42).to_parquet(f"/home/jovyan/work/chips_gb/temp/chip_bounds_per_type/{t}.parquet")
    print(t)

Create TIF chips sampled from the mosaic.

In [4]:
specs = {
    'chip_size': 32,
    'bands': [1, 2, 3], #RGB
    'mosaic_p': (
        '/home/jovyan/work/urbangrammar_samba/'
        'ghs_composite_s2/GHS-composite-S2.vrt'
    ),
    'folder': (
        '/home/jovyan/work/chips_gb/32/all/'
    ),
}

In [5]:
# max value for normalisation taken from a sample
specs['max'] = 1500

In [10]:
for t in types:
    df = geopandas.read_parquet(f"/home/jovyan/work/chips_gb/temp/chip_bounds_per_type/{t}.parquet")
    centroid = df.centroid
    df['X'] = centroid.x.astype(int)
    df['Y'] = centroid.y.astype(int)
    tools.spilled_bag_of_chips(df, specs, npartitions=16)
    print(t)

0_0
1_0
3_0
4_0
5_0
6_0
7_0
8_0
2_0
2_1
2_2
9_0
9_1
9_2
9_4
9_5


Split into train, validation and secret.

In [35]:
split = (.7, .15, .15)

for t in types:
    for subset in ["train", "validation", "secret"]:
        os.makedirs(f"{specs['folder'][:-4]}{subset}/{Path(t).stem}", exist_ok=True)
    files = glob.glob(specs["folder"] + t + "/*.tif")
    count = len(files)
    for f in tqdm(files[:int(count * split[0])]):
        f = Path(f)
        shutil.copy(f, str(f.parent.parent.parent) + "/train/" + f.parent.stem + "/" + f.name)
    for f in tqdm(files[int(count * split[0]):int(count * (split[0] + split[1]))]):
        f = Path(f)
        shutil.copy(f, str(f.parent.parent.parent) + "/validation/" + f.parent.stem + "/" + f.name)
    for f in tqdm(files[int(count * (split[0] + split[1])):]):
        f = Path(f)
        shutil.copy(f, str(f.parent.parent.parent) + "/secret/" + f.parent.stem + "/" + f.name)
    print(t, "done")

  0%|          | 0/35000 [00:00<?, ?it/s]

  0%|          | 0/7500 [00:00<?, ?it/s]

  0%|          | 0/7500 [00:00<?, ?it/s]

0_0 done


  0%|          | 0/35000 [00:00<?, ?it/s]

...

9_4 done


  0%|          | 0/1385 [00:00<?, ?it/s]

  0%|          | 0/297 [00:00<?, ?it/s]

  0%|          | 0/297 [00:00<?, ?it/s]

9_5 done
